# COVID-19 Detector

This file is a COVID-19 Detector. Previously to this file we have done a dataset_creator in Jupyter Notebook. Firsly, we upload the folders of train and test obtained with the dataset_creator to drive. Then, we start working.

Date: 2021-03-02
________________________________________________________________________________

Video: https://www.youtube.com/watch?v=nHQDDAAzIsI

Covid X-Ray Image Dataset for positive cases - https://github.com/ieee8023/covid-che...​ 

Kaggle X-Ray Chest Images for negative cases - https://www.kaggle.com/paultimothymoo...​ 

________________________________________________________________________________

**IMPORTANT:** To run this notebook it is necessary to change the runtime type to GPU.

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

First, we specify the path to the training and test folders.

In [ ]:
TRAIN_PATH = "/content/drive/MyDrive/TFM_workspace/video_coding_blocks/Dataset/train"
TEST_PATH = "/content/drive/MyDrive/TFM_workspace/video_coding_blocks/Dataset/test"

The images are very large, so we have to resize them.

Let's try to build a CNN based model in Keras. First we are going to create multiple 3 o 4 CNN layers followed by a classification.

In [ ]:
# CNN Based Model in Keras

# 4 Convolutional layers
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3), activation="relu", input_shape = (224,224,3)))
model.add(Conv2D(64,(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25)) # to find overfitting

model.add(Conv2D(64,(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Dense layers
model.add(Flatten())
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1,activation="sigmoid"))

model.compile(loss=keras.losses.binary_crossentropy, optimizer = "adam",
              metrics = ["accuracy"])

Firstly we will put our angulation layer with 32 number of filters. We are keeping the number of filters small in the beginning, because the lower layers detect features in very small part of the image. If you only see a very small part of an image, there are a very few patterns like dots, textures that you can make.

As you go deeper into the network, your receptive field of the CNN layer increases. That means the feature that it extracts is basically based on a bigger part in the original picture.

In this case we will have 32 different type of features we will extract in the first layer.

Kernel sizes: 3x3. It's a standard choice. 

We will use the REU activation.

Since this is the first layer, we need to specify the input shape, so we can specify the shape as (224,224,3).

If you add Max Pooling it is going to further increase your receptive field, so you are looking at a bigger layer.

We are not using the 5x5 filters directly, because adding two layers of 3x3 will increase the non-linearity. This will increase the non-linearity in the network and that will increase the hypothesis that we can represent more complex functions with more non-linear functions. Another reason is to reduce the number of parameters.

As I go deeper in the network, I will also increase the number of filters.

In the final layer we put a single neuron because we are performing a binary classification.

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 54, 54, 64)       

So we have 5,668,097 parameters in this model and we are going to train it from scratch.

One trend that you need to notice is that as you go deeper into the model, you need to increase the number of channels.

For the training we will use the Keras image data generator library to make the data ready for the model:

* We will rescale the data by factor 1/255. This will help us to the normalization.

* Zoom allow us to take some randon crops of the images.

In [ ]:
# Train from scratch

train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

# In the test we also do the resize
test_dataset = image.ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/TFM_workspace/video_coding_blocks/Dataset/train",
    target_size = (224,224),
    batch_size = 32,
    class_mode = "binary"
)

Found 294 images belonging to 2 classes.


In [ ]:
# What casses?
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

We have to do the same thing for validation data.

In [ ]:
validation_generator = test_dataset.flow_from_directory(
    "/content/drive/MyDrive/TFM_workspace/video_coding_blocks/Dataset/test",
    target_size = (224,224),
    batch_size = 32,
    class_mode = "binary"
)

Found 98 images belonging to 2 classes.


In [ ]:
validation_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [ ]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch = 8,
    epochs = 10,
    validation_data = validation_generator,
    validation_steps = 2
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
8/8 [==============================] - 175s 18s/step - loss: 0.9217 - accuracy: 0.5093 - val_loss: 0.6916 - val_accuracy: 0.4375
Epoch 2/10
8/8 [==============================] - 22s 3s/step - loss: 0.6690 - accuracy: 0.6663 - val_loss: 0.5918 - val_accuracy: 0.5469
Epoch 3/10
8/8 [==============================] - 11s 1s/step - loss: 0.5522 - accuracy: 0.7216 - val_loss: 0.4290 - val_accuracy: 0.9531
Epoch 4/10
8/8 [==============================] - 10s 1s/step - loss: 0.3991 - accuracy: 0.8183 - val_loss: 0.3633 - val_accuracy: 0.9219
Epoch 5/10
8/8 [==============================] - 10s 1s/step - loss: 0.3550 - accuracy: 0.8786 - val_loss: 0.1937 - val_accuracy: 0.9219
Epoch 6/10
8/8 [==============================] - 10s 1s/step - loss: 0.4196 - accuracy: 0.8040 - val_loss: 0.2367 - val_accuracy: 0.9062
Epoch 7/10
8/8 [==============================] - 10s 1s/step - loss: 0.3961 - accuracy: 0.8727 - val_loss: 0.2909 - val_accuracy: 0.9688
Epoch 8/10
8/8 [================

We have obtained an accuracy of 95.31% and a loss of 15.64% in 10 epochs.

## FALTA CONFUSION MATRIX